# Part I. ETL Pipeline for Pre-Processing the Files

## PLEASE RUN THE FOLLOWING CODE FOR PRE-PROCESSING THE FILES

#### Import Python packages 

In [1]:
# Import Python packages 
import pandas as pd
import cassandra
import re
import os
import glob
import numpy as np
import json
import csv

#### Creating list of filepaths to process original event csv data files

In [2]:
# checking your current working directory
print(os.getcwd())

# Get your current folder and subfolder event data
filepath = os.getcwd() + '/event_data'

# Create a for loop to create a list of files and collect each filepath
for root, dirs, files in os.walk(filepath):
    
# join the file path and roots with the subdirectories using glob
    file_path_list = glob.glob(os.path.join(root,'*'))
    #print(file_path_list)

/home/workspace


#### Processing the files to create the data file csv that will be used for Apache Casssandra tables

In [3]:
# initiating an empty list of rows that will be generated from each file
full_data_rows_list = [] 
    
# for every filepath in the file path list 
for f in file_path_list:

# reading csv file 
    with open(f, 'r', encoding = 'utf8', newline='') as csvfile: 
        # creating a csv reader object 
        csvreader = csv.reader(csvfile) 
        next(csvreader)
        
 # extracting each data row one by one and append it        
        for line in csvreader:
            #print(line)
            full_data_rows_list.append(line) 
            
# uncomment the code below if you would like to get total number of rows 
#print(len(full_data_rows_list))
# uncomment the code below if you would like to check to see what the list of event data rows will look like
#print(full_data_rows_list)

# creating a smaller event data csv file called event_datafile_full csv that will be used to insert data into the \
# Apache Cassandra tables
csv.register_dialect('myDialect', quoting=csv.QUOTE_ALL, skipinitialspace=True)

with open('event_datafile_new.csv', 'w', encoding = 'utf8', newline='') as f:
    writer = csv.writer(f, dialect='myDialect')
    writer.writerow(['artist','firstName','gender','itemInSession','lastName','length',\
                'level','location','sessionId','song','userId'])
    for row in full_data_rows_list:
        if (row[0] == ''):
            continue
        writer.writerow((row[0], row[2], row[3], row[4], row[5], row[6], row[7], row[8], row[12], row[13], row[16]))


In [4]:
# check the number of rows in your csv file
with open('event_datafile_new.csv', 'r', encoding = 'utf8') as f:
    print(sum(1 for line in f))

6821


# Part II. Complete the Apache Cassandra coding portion of your project. 

## Now you are ready to work with the CSV file titled <font color=red>event_datafile_new.csv</font>, located within the Workspace directory.  The event_datafile_new.csv contains the following columns: 
- artist 
- firstName of user
- gender of user
- item number in session
- last name of user
- length of the song
- level (paid or free song)
- location of the user
- sessionId
- song title
- userId

The image below is a screenshot of what the denormalized data should appear like in the <font color=red>**event_datafile_new.csv**</font> after the code above is run:<br>

<img src="images/image_event_datafile_new.jpg">

## Begin writing your Apache Cassandra code in the cells below

#### Creating a Cluster

In [5]:
from cassandra.cluster import Cluster
try:
    cluster = Cluster(['127.0.0.1'])
    # To establish connection and begin executing queries, need a session
    session = cluster.connect()
except Exception as e:
    print(e)

#### Create Keyspace

In [6]:
try:
    # dropping the old keyspace if existed
    session.execute("""DROP KEYSPACE IF EXISTS sparkifydb""")
    # creating a new instance of the keyspace
    session.execute("""CREATE KEYSPACE IF NOT EXISTS sparkifydb WITH REPLICATION =
    { 'class' : 'SimpleStrategy', 'replication_factor' : 1 }""")
except Exception as e:
    print(e)

#### Set Keyspace

In [7]:
try:
    session.set_keyspace('sparkifydb')
except Exception as e:
    print(e)

### Now we need to create tables to run the following queries. Remember, with Apache Cassandra you model the database tables on the queries you want to run.

## Create queries to ask the following three questions of the data

### 1. Give me the artist, song title and song's length in the music app history that was heard during  sessionId = 338, and itemInSession  = 4


### To achieve the best possible performance when executing this query, the sessionId column was used to partition the data across the nodes to try to make the data evenually distributed. the itemsInSession column was used to give each row in a partition unique primary key to uniquely identify rows and to optimize the performance.


In [8]:
# cretaing table session_log to answer the first query
try:
    session.execute(
    """
    CREATE TABLE IF NOT EXISTS session_log
    (
    sessionId int,
    itemInSession int,
    artist varchar,
    song varchar,
    song_length decimal,
    PRIMARY KEY(sessionId, itemInSession)
    )
    """
    )
except Exception as e:
    print(e)

In [9]:
file = 'event_datafile_new.csv'
with open(file, encoding = 'utf8') as f:
    csvreader = csv.reader(f)
    next(csvreader) # skip header
    
    for line in csvreader:
        query = "INSERT INTO session_log (sessionId, itemInSession, artist, song, song_length)"
        query = query + "VALUES (%s, %s, %s, %s, %s)"
        session.execute(query, (int(line[8]), int(line[3]), line[0], line[9], float(line[5])))

In [10]:
try:
    query  = """
    SELECT artist, song, song_length FROM session_log WHERE sessionId = 338 AND itemInSession = 4
    """
    row = session.execute(query)
    for r in row:
        print(r.artist + ", ", r.song + ", ", r.song_length)
except Exception as e:
    print(e)

Faithless,  Music Matters (Mark Knight Dub),  495.3073


### 2. Give me only the following: name of artist, song (sorted by itemInSession) and user (first and last name) for userid = 10, sessionid = 182

### Due to the big number of records that each user have, partitioning by the userid only is not enough as the the search within each partition will be inefficent. the leads to the the use of the composit partitioning key (userid,sessionid) to better distribute the data and the use of the itemInSession to uniquely identify and order the rows in every partition

In [11]:
# cretaing table user_log to answer the first query
try:
    session.execute(
    """
    CREATE TABLE IF NOT EXISTS user_log
    (
    userId int,
    sessionId int,
    itemInSession int,
    artist varchar,
    song varchar,
    firstName varchar,
    lastName varchar,
    PRIMARY KEY((userId, sessionId), itemInSession)
    )
    """
    )
except Exception as e:
    print(e)
    

In [12]:
file = 'event_datafile_new.csv'
with open(file, encoding = 'utf8') as f:
    csvreader = csv.reader(f)
    next(csvreader) # skip header
    
    for line in csvreader:
        query = "INSERT INTO user_log (userId, sessionId, itemInSession, artist, song, firstName, lastName)"
        query = query + "VALUES (%s, %s, %s, %s, %s, %s, %s)"
        session.execute(query, (int(line[10]), int(line[8]), int(line[3]), line[0], line[9], line[1], line[4]))
        
        

In [17]:
try:
    query  = """
    SELECT artist, song, firstName, lastName FROM user_log WHERE userId = 10 AND sessionId = 182
    """
    row = session.execute(query)
    for r in row:
        print(r.artist + ", ", r.song + ", ", r.firstname+ ", ", r.lastname + ", ")

except Exception as e:
    print(e)

Down To The Bone,  Keep On Keepin' On,  Sylvie,  Cruz, 
Three Drives,  Greece 2000,  Sylvie,  Cruz, 
Sebastien Tellier,  Kilometer,  Sylvie,  Cruz, 
Lonnie Gordon,  Catch You Baby (Steve Pitron & Max Sanna Radio Edit),  Sylvie,  Cruz, 


### 3. Give me every user name (first and last) in my music app history who listened to the song 'All Hands Against His Own'

###  As all the songs in the library have many listeners, partitoning the data based on the song is a good choice to evenually distribute the data across the nodes and increase the performance. uniquely identify the users "rows" in each partition the userId was used as it will also increase the performance and sort the data.

In [18]:
# cretaing table session_log to answer the first query
try:
    session.execute(
    """
    CREATE TABLE IF NOT EXISTS song_log
    (
    song varchar,
    userId int,
    firstName varchar,
    lastName varchar,
    PRIMARY KEY(song, userId)
    )
    """
    )
except Exception as e:
    print(e)

In [20]:
file = 'event_datafile_new.csv'
with open(file, encoding = 'utf8') as f:
    csvreader = csv.reader(f)
    next(csvreader) # skip header
    
    for line in csvreader:
        query = "INSERT INTO song_log (song, userId, firstName, lastname)"
        query = query + "VALUES (%s, %s, %s, %s)"
        session.execute(query, (line[9], int(line[10]), line[1], line[4]))
        
        

In [22]:
try:
    query  = """
    SELECT firstName, lastName FROM song_log WHERE song = 'All Hands Against His Own'
    """
    row = session.execute(query)
    for r in row:
         print(r.firstname + ", ", r.lastname)
            
except Exception as e:
    print(e)

Jacqueline,  Lynch
Tegan,  Levine
Sara,  Johnson


### Drop the tables before closing out the sessions

In [17]:
query = "drop table session_log"
try:
    rows = session.execute(query)
except Exception as e:
    print(e)

query = "drop table user_log"
try:
    rows = session.execute(query)
except Exception as e:
    print(e)

query = "drop table song_log"
try:
    rows = session.execute(query)
except Exception as e:
    print(e)

### Close the session and cluster connection¶

In [18]:
session.shutdown()
cluster.shutdown()